In [101]:
import pandas as pd
import numpy as np

data = pd.read_csv('/home/nika/code/marcnaweb/car_recommendation_engine/raw_data/car_files_4c_en.csv')
df = data.copy()
df.head()

/tmp/ipykernel_3350/3543565571.py:4: DtypeWarning: Columns (24,25,26,35,43,53,59,63,74,75,85,89,108,110,116,120,122,124,136,141,147,150,151,162,163,177,178,180,195,214,221,226,234,241,246,259,266,270,271,273,278,283,287,290) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/home/nika/code/marcnaweb/car_recommendation_engine/raw_data/car_files_4c_en.csv')


,Unnamed: 0.1,Unnamed: 0,Hot air,Volumetric anti -first alarm,Harman Kardon Sound System,Track change alert,Carbonaceramic brake disc,Arconditional Zones 2,Reclinarinar rear seat,Maximum rotation,...,Folding rear seat,Free wheel,Shaker Pro Audio Sound System,Maximum speed electric mode,Electronic noise cancellation,Steering wheel adjustment in depth,Facial recognition camera,Carcode,Perimeter anti theft alarm,Track centralization assistant
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN


## Too many features with NaN's, remove columns which have more then 15000 Nan'n in column

In [177]:
# Rename Carcode column to car_code (we have another df, it should have same name)
df.rename(columns={'Carcode': 'car_code'}, inplace=True)
# Set car_code as index
df.set_index('car_code', inplace=True)

filtered_columns = df.columns[df.isna().sum() < 15000]
filtered_df = df[filtered_columns]
# filtered_df

## Drop some rows with a lot of Nan's

In [178]:
test_filtered_df =filtered_df.dropna(subset='Acceleration')
# test_filtered_df

## Check how much % is NaN in every column and check Dtype

In [179]:
nan_percentage_per_column = test_filtered_df.isnull().mean() * 100

# Create a new DataFrame with column names and NaN percentages and Dtypes
nan_info_df = pd.DataFrame({
    'Column Name': nan_percentage_per_column.index,
    'NaN Percentage': nan_percentage_per_column.values,
    'Data Type': test_filtered_df.dtypes
})

# Display the new DataFrame
nan_info_df # To check whole list: nan_info_df[0:50] and nan_info_df[50:100]

,Column Name,NaN Percentage,Data Type
Unnamed: 0.1,Unnamed: 0.1,0.000000,int64
Hot air,Hot air,46.233340,object
Acceleration,Acceleration,0.000000,object
Length,Length,0.000000,object
Maximum torque,Maximum torque,0.709871,object
...,...,...,...
Platform,Platform,42.944756,object
Year,Year,0.125555,float64
Radio,Radio,49.565385,object
Folding rear seat,Folding rear seat,52.796021,object


## After analysing all features, decided to remove some of them.
Neither they don't give information, or they are hard to preprocess.


In [180]:
column_to_remove = ['Unnamed: 0.1','Assistance','Aspiration', 'Rear tires', 'Spare tire', 'Front tires', 'Tuching',
                    'Urban autonomy', 'Generation', 'Front suspension', 'Rear suspension', 'Coupling', 'Valve command',
                    'Places', 'Disposition', 'Cylinders', 'Elastic element', 'Ipva R', 'Frontal area A', 'Engine code',
                    'Traction', 'Installation', 'Road autonomy', 'Engine power supply', 'Engine control activation',
                    'Gear change code', 'Corrected frontal area', 'Platform']


In [181]:
new_filtered_df = test_filtered_df.drop(columns=column_to_remove)
len(new_filtered_df.columns)

70

# Preprocessing Features

## Preprocessing All features with [Nan, standart equipment, optinal equipment]
saved in: columns_to_transform ---------
Nan = 0 ----------------
standart equipment, optinal equipment = 1

In [182]:
preprocessing_df = new_filtered_df.copy()

columns_to_transform = ['Hot air', 'Rev counter', 'Assisted direction', 'ABS brakes', 'Rear window', 'Central locking of the doors',
                        'Headrest for all occupants', 'Electric rearview mirror adjustment', 'Air conditioning',
                        'Bluetooth connection', 'Frontal Airbags', 'Steering wheel adjustment height',
                        'Electric front window control', 'Multifunctional steering wheel', 'Driver s seat with height adjustment',
                        'On board computer', 'Light in the trunk', 'Alloy wheels', 'USB connection', 'Radio',
                        'Folding rear seat', 'Perimeter anti theft alarm', 'Cooling liquid thermometer']

# Replace NaN with 0 and other values with 1
preprocessing_df[columns_to_transform] = preprocessing_df[columns_to_transform].applymap(lambda x: 0 if pd.isnull(x) else 1)


/tmp/ipykernel_3350/589099134.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  preprocessing_df[columns_to_transform] = preprocessing_df[columns_to_transform].applymap(lambda x: 0 if pd.isnull(x) else 1)


## Preprocessing [Acceleration]

In [183]:
# Preprocess Acceleration (0100 km/h 3,8 s = 3.8)
preprocessing_df['Acceleration'] = preprocessing_df['Acceleration'].str.extract(r'(\d+\,\d+)')

# Replace ',' with '.' and convert to numeric
preprocessing_df['Acceleration'] = preprocessing_df['Acceleration'].str.replace(',', '.').astype(float)
# Rename Column
# preprocessing_df = preprocessing_df.rename(columns={'Acceleration': 'Acceleration 0100 km/h in S'})

## Preprocessing [All features with strings]

In [9]:
# Rename columns
# preprocessing_df = preprocessing_df.rename(columns={'Length': 'Length mm',
#                                                     'Width': 'Width mm',
#                                                     'Maximum torque': 'Maximum torque kgfm'})

In [184]:
def extract_float_value(value):
    try:
        if isinstance(value, float):
            return value
        else:
            float_value = value.split()[0].replace(',', '.')
            return float(float_value)
    except (ValueError, IndexError):
        return np.nan

for column in preprocessing_df[['Weight/Torque', 'Weight', 'Weight/power', 'Max power regime.', 'Cylinder diameter',
                      'Fuel tank', 'Specific power', 'Maximum power', 'Length', 'Maximum torque', 'Width', 'Height',
                      'Specific torque', 'Minimum height from the ground', 'Piston course', 'Front gauge', 'Displacement',
                      'Turns diameter', 'Rear gauge', 'Length between the axis', 'Maximum speed', 'Road consumption',
                      'Max torque regime', 'Car payload', 'Sidewall height', 'Unit displacement', 'Trunk', 'Urban',
                      'Guarantee', 'Reader score', 'Compression ratio', 'Drag coefficient', 'Price R', 'Devaluation',
                      'CNW Index']]:
    preprocessing_df[column] = preprocessing_df[column].apply(extract_float_value)


In [185]:
len(preprocessing_df.columns)

70

## Creating a pipeline

In [186]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [187]:
def binarize_column(column):
    binarizer = LabelBinarizer()
    return binarizer.fit_transform(column)

In [188]:
columns_to_encode = ['Propulsion', 'Car gearbox', 'Fuel', 'Settings', 'Car size']

numeric_columns = preprocessing_df.select_dtypes(include=['number'])

numeric_column_names = numeric_columns.columns.tolist()
len(numeric_column_names)

64

In [189]:
# pipelne for whole df // WORKING !!!!

pipeline = Pipeline([
    ('preprocessor', ColumnTransformer([
        ('onehot', OneHotEncoder(), columns_to_encode),  # Apply one-hot encoding to specified columns
        ('binarize', FunctionTransformer(binarize_column, validate=False), ['Provenance']),  # Binarize the binary column
        ('imputer', SimpleImputer(strategy='median'), numeric_column_names),  # Impute missing values
    ], remainder='passthrough')),
    ('scaler', MinMaxScaler())
])

In [190]:
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehot', OneHotEncoder(),
                                                  ['Propulsion', 'Car gearbox',
                                                   'Fuel', 'Settings',
                                                   'Car size']),
                                                 ('binarize',
                                                  FunctionTransformer(func=<function binarize_column at 0x7f171eb3dfc0>),
                                                  ['Provenance']),
                                                 ('imputer',
                                                  SimpleImputer(strategy='median'),
                                                  ['Hot air', 'Acceleration',
                                                   'Length', 'Maxi...
                                                   'Drag coefficient',
                                                   'Central locking of the '
                                                   'doors',
                                                   'Height',
                                                   'Valves per cylinder',
                                                   'Specific torque',
                                                   'Minimum height from the '
                                                   'ground',
                                                   'Piston course', 'Weight',
                                                   'Price R',
                                                   'Headrest for all occupants',
                                                   'Front gauge',
                                                   'Electric rearview mirror '
                                                   'adjustment',
                                                   'Weight/power',
                                                   'Displacement',
                                                   'Cooling liquid thermometer',
                                                   'Max power regime.',
                                                   'Turns diameter', ...])])),
                ('scaler', MinMaxScaler())])

In [213]:
testing_pipe_df = preprocessing_df.copy()

testing_pipe_df_end = pipeline.fit_transform(testing_pipe_df)



# Get the names of the transformed features after one-hot encoding
encoded_feature_names = pipeline.named_steps['preprocessor'].named_transformers_['onehot'].get_feature_names_out(input_features=columns_to_encode)

# Get the names of the remaining columns
remaining_column_names = list(pipeline.named_steps['preprocessor'].transformers_[-1][2])  # Get the names of passthrough columns

# Combine all column names
all_column_names = list(encoded_feature_names) + ['Provenance'] + remaining_column_names



testing_pipe_df_end = pd.DataFrame(testing_pipe_df_end, columns=all_column_names)

testing_pipe_df_end.index = preprocessing_df.index

testing_pipe_df_end

,Propulsion_Combustion,Propulsion_Electric,Propulsion_Hybrid,Propulsion_Light Hybrid,Propulsion_Plug-in hybrid,Car gearbox_Automated,Car gearbox_Automatic,Car gearbox_CVT,Car gearbox_Manual,Fuel_Alcohol,...,Maximum power,Unit displacement,Trunk,USB connection,Gear speed transmissions,Urban,Year,Radio,Folding rear seat,Perimeter anti theft alarm
car_code,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.302210,0.517060,0.003001,0.0,0.545455,0.002481,0.607143,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.012707,0.249986,0.019006,0.0,0.454545,0.191067,0.607143,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.046409,0.500995,0.024341,0.0,0.454545,0.121588,0.607143,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.065193,0.445773,0.021674,0.0,0.454545,0.111663,0.607143,0.0,0.0,0.0
6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.055249,0.437741,0.024341,0.0,0.454545,0.166253,0.607143,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23884,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.048619,0.333321,0.019673,1.0,0.545455,0.148883,1.000000,1.0,1.0,1.0
23885,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.048619,0.333321,0.019673,1.0,0.545455,0.148883,1.000000,1.0,1.0,1.0
23893,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.048619,0.333321,0.019673,1.0,0.545455,0.166253,1.000000,1.0,1.0,1.0


In [ ]:
testing_pipe_df_end.to_csv("scaled_cleaned.csv")